<a href="https://colab.research.google.com/github/elenalu19/Airplane_Accident_Analysis_2000-2024/blob/main/Aviation_Data_Collect%26Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf
from pypdf import PdfReader
import requests
import tempfile # For creating temporary files
import os       # For path manipulation
import pandas as pd
import re
import time

# Install gspread and google-auth
!pip install gspread google-auth

# Authenticate with Google Colab
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Publish Sheet to Web, retrieve public CSV URL, utiliize pandas to read csv
google_sheet_csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vScIM9RwQkLPP1SHe-XPJnW3hJpSu-0SU31m5KrNL-7APpc1yBW0SShElQ1EfzFyaCAA7e3muYI98_T/pub?gid=6524437&single=true&output=csv'
try:
  df = pd.read_csv(google_sheet_csv_url)
   # Display the first few rows to verify
 # print(df.head())
# Create notificatiion for potential error
except Exception as e:
    print(f"Error reading the CSV from the URL: {e}")

In [ ]:
df.head()

,NtsbNo,EventDate,City,State,Country,HasSafetyRec,HighestInjuryLevel,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,...,AirCraftDamage,WeatherCondition,Operator,BroadPhaseofFlight,RepGenFlag,MostRecentReportType,DocketUrl,ReportUrl,pilot age,pilot flight experience
0,NYC00IA100,2000-03-24T15:20:00Z,BOSTON,Massachusetts,United States,False,NaN,0.0,0.0,0.0,...,"Minor, Minor",VMC,Delta Air Lines,NaN,NaN,Final,NaN,https://data.ntsb.gov/carol-repgen/api/Aviatio...,52,"13000 hours (Total, all aircraft), 5979 hours ..."
1,NYC00LA086,2000-03-01T04:47:00Z,NEWARK,New Jersey,United States,False,NaN,0.0,0.0,0.0,...,Substantial,VMC,Federal Express,NaN,NaN,Final,NaN,https://data.ntsb.gov/carol-repgen/api/Aviatio...,55,"6842 hours (Total, all aircraft), 4745 hours (..."
2,NYC00IA078,2000-02-12T19:45:00Z,BOSTON,Massachusetts,United States,False,Minor,0.0,0.0,2.0,...,Minor,VMC,NaN,NaN,NaN,Final,NaN,https://data.ntsb.gov/carol-repgen/api/Aviatio...,52,Information Not in PDF
3,NYC00LA097,2000-02-03T09:00:00Z,BOSTON,Massachusetts,United States,False,Serious,0.0,1.0,0.0,...,NaN,VMC,US Airways,NaN,NaN,Final,NaN,https://data.ntsb.gov/carol-repgen/api/Aviatio...,47,"13000 hours (Total, all aircraft), 4700 hours ..."
4,MIA00LA206,2000-07-02T11:42:00Z,ORLANDO,Florida,United States,False,NaN,0.0,0.0,0.0,...,Substantial,VMC,"PRO AIR, INC.",NaN,NaN,Final,NaN,https://data.ntsb.gov/carol-repgen/api/Aviatio...,52,"12920 hours (Total, all aircraft), 1844 hours ..."


In [ ]:
# # Convert the 'EventDate' column to datetime objects
# df['EventDate'] = pd.to_datetime(df['EventDate'], errors='coerce')

# # Extract the day of the week as a new column
# # Use .dt.day_name() for the full name (e.g., "Monday")
# df['EventDayOfWeek'] = df['EventDate'].dt.day_name()

# # Display the first few rows of the DataFrame with the new column
# print(df[['EventDate', 'EventDayOfWeek']].head())

# # Optionally, export the DataFrame to a CSV file
# # df.to_csv('dates_and_days.csv', index=False)

   EventDate EventDayOfWeek
0 2000-03-24         Friday
1 2000-03-01      Wednesday
2 2000-02-12       Saturday
3 2000-02-03       Thursday
4 2000-07-02         Sunday


In [ ]:
# # Add the 'EventDayOfWeek' column to the Google Sheet
# spreadsheet_name = 'ext_far121_ntsb' # Replace with your spreadsheet name
# sheet_name = 'Sheet1' # Replace with your sheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(sheet_name)

#     # Get the values from the 'EventDayOfWeek' column in your DataFrame
#     day_of_week_values = df['EventDayOfWeek'].values.tolist()

#     # Prepare the data to upload. It should be a list of lists, with each inner list
#     # representing a row in the sheet. We'll add a header for the new column.
#     data_to_upload = [['EventDayOfWeek']] + [[day] for day in day_of_week_values]

#     # Specify the column where you want to add this data. Column 4 is 'D'.
#     start_cell = 'D1'


#     # Update the sheet with the new column data
#     worksheet.update(values=data_to_upload, range_name=start_cell)

#     print(f"Successfully added the 'EventDayOfWeek' column to the Google Sheet starting at cell {start_cell}.")

# except gspread.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found.")
# except gspread.WorksheetNotFound:
#     print(f"Worksheet '{sheet_name}' not found.")
# except Exception as e:
#     print(f"An error occurred while updating the Google Sheet: {e}")

Successfully added the 'EventDayOfWeek' column to the Google Sheet starting at cell D1.


In [ ]:
# # Parsing through PDFs to find airline/operators
# # Assuming 'df' is already loaded and contains your data

# def extract_operator_from_pdf(pdf_url):
#     """
#     Downloads a PDF from a URL, extracts text, and searches for the operator name.

#     Args:
#         pdf_url (str): The URL of the PDF report.

#     Returns:

#         str: The extracted operator name, or "Information Not Found" if not found
#              or an error occurred.
#     """
#     if pd.isna(pdf_url):
#         print("Skipping due to missing ReportUrl")
#         return "Missing URL"

#     try:
#         response = requests.get(pdf_url, stream=True)
#         response.raise_for_status()

#         if 'application/pdf' not in response.headers.get('Content-Type', ''):
#             print(f"Skipping as the URL does not point to a PDF: {pdf_url}")
#             response.close()
#             return "Not a PDF"

#         with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as temp_file:
#             for chunk in response.iter_content(chunk_size=8192):
#                  temp_file.write(chunk)
#             temp_file_path = temp_file.name

#         response.close()

#         reader = PdfReader(temp_file_path)
#         full_text = ""

#         for page_num in range(0, len(reader.pages)):
#             page = reader.pages[page_num]
#             try:
#                 page_text = page.extract_text()
#                 if page_text:
#                     full_text += page_text + "\n"
#             except Exception as e:
#                  print(f"Could not extract text from page {page_num + 1}: {e}")
#                  continue

#         os.remove(temp_file_path)

#         # Modified regex to handle cases with preceding comma and "as flight"
#         # Updated regex to explicitly account for newlines between "operated" and "by" or "as"
#         operator_match = re.search(r"(?:,|\.)?\s*operated[\s\n]+(?:by|as)[\s\n]*(.*?)(?:,|\.|as flight|flight)", full_text, re.IGNORECASE)


#         if operator_match:
#             operator_name = operator_match.group(1).strip()
#             return operator_name
#         else:
#             # Print full_text for debugging when "Information Not Found"
#             print("\n--- Full text from PDF when 'Information Not Found' ---")
#             print(full_text)
#             print("----------------------------------------------------")
#             return "Information Not Found" # Specific message if text is extracted but pattern not found

#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading the PDF from {pdf_url}: {e}")
#         return "Download Error"

#     except Exception as e:
#         print(f"An error occurred while processing {pdf_url}: {e}")
#         return "Processing Error"

# # Iterate through the DataFrame and process rows with empty or "Information Not Found" Operator cells
# total_rows = len(df)
# newly_extracted_count = 0 # Initialize a counter

# for index, row in df.iterrows():
#     # Check if the 'Operator' column is empty, NaN, empty string, or "Information Not Found"
#     if pd.isna(row['Operator']) or row['Operator'] == "" or row['Operator'] == "Information Not Found" or row['Operator'] == "Not Available":
#         pdf_url = row['ReportUrl']
#         # print(f"Processing row {index + 1}/{total_rows} (URL: {pdf_url}) - Operator is empty")
#         extracted_operator = extract_operator_from_pdf(pdf_url)

#         # Check if a valid operator was extracted (not any of the error/not found messages)
#         if extracted_operator not in ["Information Not Found", "Missing URL", "Not a PDF", "Download Error", "Processing Error"]:
#             df.at[index, 'Operator'] = extracted_operator
#             newly_extracted_count += 1 # Increment counter for successful extractions

#         print(f"Row {index + 2}/{total_rows}: Extracted Operator: {extracted_operator}")
#         # Add a small delay to avoid hitting rate limits
#         time.sleep(4)
#     #else:
#         #print(f"Skipping row {index + 1}/{total_rows} (Operator: {row['Operator']}) - Operator already present")


# # Display the first few rows to see the updated 'Operator' column
# print("\nDataFrame after processing:")
# display(df.head())

# # Print the total count of newly extracted operators
# print(f"\nTotal number of newly extracted operators: {newly_extracted_count}")

Error downloading the PDF from https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/51265/pdf: 403 Client Error: Forbidden for url: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/51265/pdf
Row 4/1099: Extracted Operator: Download Error


KeyboardInterrupt: 

In [ ]:
# # Testing Single PDF URL to refine code for operator informatio.
# # Define a test URL
# test_url = 'https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/64632/pdf' # Replace with a URL you want to test

# # Process the test URL and get the extracted operator
# extracted_operator = extract_operator_from_pdf(test_url)

# # Print the extracted operator
# print(f"Extracted Operator for {test_url}: {extracted_operator}")

# # You can also inspect the function's behavior by adding print statements inside extract_operator_from_pdf

NameError: name 'extract_operator_from_pdf' is not defined

In [ ]:
# # Check for empty operator column and update with parsed operator
# # Open the spreadsheet and the desired sheet
# spreadsheet_name = 'ext_far121_ntsb' # Replace with your spreadsheet name
# sheet_name = 'Sheet1' # Replace with your sheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(sheet_name)

#     # Get all values from the 'Operator' column (Column AB is 28th column, index 27)
#     # We'll get the header row and all data rows
#     operator_column_values = worksheet.col_values(26) # Column AB is the 28th column


#     # Assuming your DataFrame 'df' is already updated with the extracted operator names

#     # Prepare a list of updates to send to the sheet
#     # This will be a list of lists, where each inner list is [row, col, value]
#     updates = []

#     # Start from the second row (index 1) to skip the header
#     for i in range(1, len(operator_column_values)):
#         sheet_operator_value = operator_column_values[i]
#         df_operator_value = df.at[i-1, 'Operator'] # Get the corresponding value from your DataFrame (df index is 0-based)

#         # Replace NaN with "Not Available" in the DataFrame value before checking and adding to updates
#         if pd.isna(df_operator_value):
#             df_operator_value = "Not Available"

#         # Check if the sheet cell was empty and the DataFrame cell is NOT empty and not "Information Not Found" or other error messages
#         if (sheet_operator_value == "" or sheet_operator_value is None or sheet_operator_value == "Not Available") and \
#            (df_operator_value is not None and df_operator_value != "" and \
#             df_operator_value != "Information Not Found" and \
#             df_operator_value != "Missing URL" and \
#             df_operator_value != "Not a PDF" and \
#             df_operator_value != "Download Error" and \
#             df_operator_value != "Processing Error"):

#             # If the sheet cell was empty and df has a valid extracted value, add it to updates
#             # Row index in gspread is 1-based, column index is 1-based
#             updates.append({'range': f'Z{i + 1}', 'values': [[df_operator_value]]})


#     # If there are updates, send them to the sheet
#     if updates:
#         worksheet.batch_update(updates)
#         print(f"Updated {len(updates)} empty 'Operator' cells in the Google Sheet.")
#     else:
#         print("No empty 'Operator' cells found to update or no valid information extracted.")


# except gspread.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found.")
# except gspread.WorksheetNotFound:
#     print(f"Worksheet '{sheet_name}' not found.")
# except Exception as e:
#     print(f"An error occurred while updating the Google Sheet: {e}")

Updated 269 empty 'Operator' cells in the Google Sheet.


In [ ]:
# #Clean the 'Operator' column and update Google Sheet
# df['Operator'] = df['Operator'].astype(str).str.upper()
# df['Operator'] = df['Operator'].str.replace('AIR LINES', 'AIRLINES', regex=False)
# df['Operator'] = df['Operator'].str.replace(r'\s*INC\.|\s*,\s*INC\.|\s*CO\b|\s*CORP\b|\s*INC\b', '', regex=True)
# df['Operator'] = df['Operator'].str.strip()
# df['Operator'] = df['Operator'].str.replace(r'[.,]$', '', regex=True) # Remove trailing periods or commas


# # # Display the updated 'Operator' column to verify
# # print(df['Operator'].to_string())

# # Open the spreadsheet and the desired sheet
# spreadsheet_name = 'ext_far121_ntsb' # Replace with your spreadsheet name
# sheet_name = 'Sheet3' # Replace with your sheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(sheet_name)

#     # Get all values from the 'Operator' column (Column AB is 28th column, index 27)
#     # We'll get the header row and all data rows
#     operator_column_values = worksheet.col_values(3) # Column AB is the 28th column

#     # Prepare a list of updates to send to the sheet
#     # This will be a list of lists, where each inner list is [row, col, value]
#     updates = []

#     # Start from the second row (index 1) to skip the header
#     for i in range(1, len(operator_column_values)):
#         sheet_operator_value = operator_column_values[i]
#         df_operator_value = df.at[i-1, 'Operator'] # Get the corresponding value from your DataFrame (df index is 0-based)

#         # We will update the sheet with the value from the DataFrame regardless of the current sheet value
#         updates.append({'range': f'C{i + 1}', 'values': [[df_operator_value]]})


#     # If there are updates, send them to the sheet
#     if updates:
#         worksheet.batch_update(updates)
#         print(f"Updated {len(updates)} 'Operator' cells in the Google Sheet.")
#     else:
#         print("No 'Operator' cells found to update or no valid information extracted.")


# except gspread.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found.")
# except gspread.WorksheetNotFound:
#     print(f"Worksheet '{sheet_name}' not found.")
# except Exception as e:
#     print(f"An error occurred while updating the Google Sheet: {e}")

Updated 1099 'Operator' cells in the Google Sheet.


In [ ]:
# #Add the new columns to the DataFrame
# df['pilot age'] = None
# df['pilot flight experience'] = None

In [ ]:
# # Iterating through rows of PDF URLs, parse through text, extract pilot information
# # create variable for column "ReportUrl"
# report_links = df.ReportUrl
# # Get the total number of rows
# total_rows = len(df)

# # Iterate through report URLs
# for index, pdf_url in report_links.items():
#     # Print the current progress
#     print(f"Processing row {index + 1}/{total_rows} (URL: {pdf_url})")

#     if pd.isna(pdf_url):
#         print(f"Skipping row {index} due to missing ReportUrl")
#         # Set values to "Not Available" for rows with missing URLs
#         df.at[index, 'pilot age'] = "Not Available"
#         df.at[index, 'pilot flight experience'] = "Not Available"
#         continue

#     try:
#         # retrieves the content of url and stores it in response
#         response = requests.get(pdf_url, stream=True) # Use stream=True for potentially large files
#         # Raise an exception for HTTP errors
#         response.raise_for_status()

#         # Check if the content type is PDF
#         if 'application/pdf' not in response.headers.get('Content-Type', ''):
#             print(f"Skipping row {index} as the URL does not point to a PDF: {pdf_url}")
#             # Set values to "Not Available" for non-PDF URLs
#             df.at[index, 'pilot age'] = "Not Available"
#             df.at[index, 'pilot flight experience'] = "Not Available"
#             response.close() # Close the response stream
#             continue


#         # delete=False to prevent the automatic deletion, suffix='.pdf' to ensure
#         # working with pdf
#         with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as temp_file:
#             # transports the content of response into tempfile
#             # Use response.iter_content for streaming downloads
#             for chunk in response.iter_content(chunk_size=8192):
#                  temp_file.write(chunk)
#             # .name is an attribute of the object tempfile
#             # save that attribute to a variable to cleanly access later using PdfReader
#             temp_file_path = temp_file.name

#         response.close() # Close the response stream after writing to file

#         reader = PdfReader(temp_file_path)
#         full_text = "" # Extract all text first

#         for page_num in range(0, len(reader.pages)):
#             page = reader.pages[page_num]
#             try:
#                 page_text = page.extract_text()
#                 if page_text:
#                     full_text += page_text + "\n" # Add page text and a newline
#             except Exception as e:
#                  print(f"Could not extract text from page {page_num + 1} of {pdf_url}: {e}")
#                  continue

#         os.remove(temp_file_path)

#         # Define regular expressions to find sections and data within pages
#         pilot_section_start_pattern = r"Pilot Information"
#         section_end_keywords = ["Co-pilot Information", "Airplane and", "Flight engineer", "Other flight", "The Captain", "Cabin crew", "Aircraft and", "Page"]
#         flight_time_pattern = r"Flight Time:"
#         age_pattern = r"Age:\s*(\d+)"

#         pilot_text = ""

#         # Extract Pilot Information
#         pilot_match_start = re.search(pilot_section_start_pattern, full_text)
#         if pilot_match_start:
#             pilot_text_start_index = pilot_match_start.end()
#             pilot_text_end_index = len(full_text) # Default to end of text

#             # Find the end of the pilot section based on keywords
#             for keyword in section_end_keywords:
#                 keyword_match = re.search(re.escape(keyword), full_text[pilot_text_start_index:], re.IGNORECASE)
#                 if keyword_match:
#                     current_end_index = pilot_text_start_index + keyword_match.start()
#                     if current_end_index < pilot_text_end_index:
#                         pilot_text_end_index = current_end_index

#             pilot_text = full_text[pilot_text_start_index:pilot_text_end_index].strip()

#             # Extract Age and Flight Time from collected pilot text
#             if pilot_text:
#                 pilot_ages = re.findall(age_pattern, pilot_text)
#                 # Adjusted regex to capture text after "Flight Time:" and handle potential variations
#                 pilot_flight_times = re.findall(flight_time_pattern + r"\s*(.*)", pilot_text, re.S)

#                 if pilot_ages:
#                     df.at[index, 'pilot age'] = pilot_ages[0]
#                 else:
#                      df.at[index, 'pilot age'] = "Not Available" # Set if section found but age not

#                 if pilot_flight_times:
#                     df.at[index, 'pilot flight experience'] = " ".join(pilot_flight_times).strip()
#                 else:
#                      df.at[index, 'pilot flight experience'] = "Not Available" # Set if section found but flight time not


#         else:
#             # Set values to "Not Available" if pilot section was not found in pages
#             df.at[index, 'pilot age'] = "Information Not in PDF" # Changed to reflect section not found
#             df.at[index, 'pilot flight experience'] = "Information Not in PDF" # Changed to reflect section not found

#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading the PDF from {pdf_url}: {e}")
#         # Set values to "Error processing PDF" for rows with download errors
#         df.at[index, 'pilot age'] = "Error processing PDF"
#         df.at[index, 'pilot flight experience'] = "Error processing PDF"

#     except Exception as e:
#         print(f"An error occurred while processing {pdf_url}: {e}")
#         # Set values to "Error processing PDF" for rows with other processing errors
#         df.at[index, 'pilot age'] = "Error processing PDF"
#         df.at[index, 'pilot flight experience'] = "Error processing PDF"
#     # Add a small delay to avoid hitting rate limits
#     time.sleep(3)

Processing row 1/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/195189/pdf)
Processing row 2/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194995/pdf)
Processing row 3/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194881/pdf)
Processing row 4/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194882/pdf)
Processing row 5/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194848/pdf)
Processing row 6/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194782/pdf)
Processing row 7/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194718/pdf)
Processing row 8/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/194158/pdf)
Processing row 9/1099 (URL: https://data

In [ ]:
# # Import parsed information into Google Sheet
# # Open the spreadsheet and the desired sheet
# spreadsheet_name = 'ext_far121_ntsb' # Replace with your spreadsheet name
# sheet_name = 'Sheet1' # Replace with your sheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(sheet_name)

#     # Select only the new columns
#     new_columns_df = df[['pilot age', 'pilot flight experience']].copy()

#     # Replace None and NaN values with "Not Available" for Google Sheet compatibility in the new columns
#     new_columns_df_cleaned = new_columns_df.fillna("Not Available")
#     new_columns_df_cleaned = new_columns_df_cleaned.replace({None: "Not Available"})

#     # Convert DataFrame of new columns to a list of lists for gspread
#     # Include headers for the new columns
#     data_to_upload = [new_columns_df_cleaned.columns.values.tolist()] + new_columns_df_cleaned.values.tolist()


#     # Determine the starting cell for the new columns (AU1)
#     start_cell = 'AU1'

#     # Update the sheet with only the new columns
#     worksheet.update(values=data_to_upload, range_name=start_cell)

# except gspread.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found.")
# except gspread.WorksheetNotFound:
#     print(f"Worksheet '{sheet_name}' not found.")
# except Exception as e:
#     print(f"An error occurred while updating the Google Sheet: {e}")

In [ ]:
# # Iterate through rows of PDF URLs, processing only those with "Not Available" or "Error processing PDF" for pilot info
# report_links = df.ReportUrl
# total_rows = len(df)

# # Iterate through report URLs
# for index, pdf_url in report_links.items():
#     # Check if pilot age or pilot flight experience is "Not Available" or "Error processing PDF"
#     if df.at[index, 'pilot age'] in ["Not Available"] or \
#        df.at[index, 'pilot flight experience'] in ["Not Available"]:

#         print(f"Re-processing row {index + 1}/{total_rows} (URL: {pdf_url}) due to missing or errored pilot info")

#         # Use the function to process the single PDF URL
#         extracted_data = process_single_pdf_url(pdf_url)

#         # Update the DataFrame with the extracted data
#         df.at[index, 'pilot age'] = extracted_data['pilot age']
#         df.at[index, 'pilot flight experience'] = extracted_data['pilot flight experience']

#         # Add a small delay to avoid hitting rate limits
#         time.sleep(3)
#     #else:
#         #print(f"Skipping row {index + 1}/{total_rows} (URL: {pdf_url}) - Pilot info already present")


# print("\nDataFrame after re-processing:")
# display(df[['ReportUrl', 'pilot age', 'pilot flight experience']].head())

Re-processing row 37/1099 (URL: https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/192290/pdf) due to missing or errored pilot info

--- Full Text from PDF ---
Page 1 of 7
Aviation Investigation Final Report
Location: Houston, Texas Accident Number: DCA23LA304
Date & Time: June 1, 2023, 16:32 Local Registration: N479UA
Aircraft: AIRBUS INDUSTRIE A320-232 Aircraft Damage: None
Defining Event: Turbulence encounter Injuries: 1 Serious, 117 None
Flight Conducted Under: Part 121: Air carrier - Scheduled
Analysis 
A flight attendant was seriously injured when United Airlines flight 1288 encountered low-topped 
convectively-induced turbulence during a descent into the George Bush Intercontinental 
Houston Airport (IAH), Houston, Texas.
Flight 1288 originated from Baltimore/Washington International Thurgood Marshall Airport 
(BWI), Baltimore, Maryland.  The captain was the pilot monitoring, and the first officer was the 
pilot flying.
The flight crewmembers reporte

,ReportUrl,pilot age,pilot flight experience
0,https://data.ntsb.gov/carol-repgen/api/Aviatio...,49,"16899 hours (Total, all aircraft), 1440 hours ..."
1,https://data.ntsb.gov/carol-repgen/api/Aviatio...,48,"(Estimated) 7000 hours (Total, all aircraft), ..."
2,https://data.ntsb.gov/carol-repgen/api/Aviatio...,37,"7500 hours (Total, all aircraft), 1657 hours (..."
3,https://data.ntsb.gov/carol-repgen/api/Aviatio...,64,"30000 hours (Total, all aircraft), 13000 hours..."
4,https://data.ntsb.gov/carol-repgen/api/Aviatio...,51,"1177 hours (Total, this make and model)"


In [ ]:
# # Define a function to process a single PDF URL
# def process_single_pdf_url(pdf_url):
#     """
#     Processes a single PDF URL to extract pilot information.

#     Args:
#         pdf_url (str): The URL of the PDF report.

#     Returns:
#         dict: A dictionary containing extracted information. Values will be
#               "Information Not in PDF" if the respective section is not found
#               within a successfully downloaded PDF, "Error processing PDF" if
#               there was a download or processing error, "Not Available" if the
#               section is found but the specific data (age or flight time) is
#               not present within the section, or the extracted data if successful.
#     """
#     result = {
#         'pilot age': "Information Not in PDF", # Default if section not found
#         'pilot flight experience': "Information Not in PDF", # Default if section not found
#         'pilot_text': "", # Add pilot_text to the result dictionary
#     }

#     if pd.isna(pdf_url):
#         print("Skipping due to missing ReportUrl")
#         # For missing URLs, let's use a specific message
#         result['pilot age'] = "Missing URL"
#         result['pilot flight experience'] = "Missing URL"
#         return result

#     try:
#         response = requests.get(pdf_url, stream=True)
#         response.raise_for_status()

#         if 'application/pdf' not in response.headers.get('Content-Type', ''):
#             print(f"Skipping as the URL does not point to a PDF: {pdf_url}")
#             # For non-PDF URLs, let's use a specific message
#             result['pilot age'] = "Not a PDF"
#             result['pilot flight experience'] = "Not a PDF"
#             response.close()
#             return result

#         with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as temp_file:
#             for chunk in response.iter_content(chunk_size=8192):
#                  temp_file.write(chunk)
#             temp_file_path = temp_file.name

#         response.close()

#         reader = PdfReader(temp_file_path)
#         full_text = "" # Extract all text first

#         for page_num in range(0, len(reader.pages)):
#             page = reader.pages[page_num]
#             try:
#                 page_text = page.extract_text()
#                 if page_text:
#                     full_text += page_text + "\n" # Add page text and a newline
#             except Exception as e:
#                  print(f"Could not extract text from page {page_num + 1}: {e}")
#                  continue

#         os.remove(temp_file_path)

#         #Print the full text for debugging
#         print("\n--- Full Text from PDF ---")
#         print(full_text)
#         print("--------------------------\n")

#         # Define regular expressions to find sections and data within pages
#         pilot_section_start_pattern = r"Pilot Information"
#         # Adjusted section_end_keywords to include "Co-pilot Information"
#         section_end_keywords = ["Co-pilot Information", "Airplane and", "Flight engineer", "Other flight", "The Captain", "Cabin crew", "Aircraft and", "Page"]
#         flight_time_pattern = r"Flight Time:"
#         age_pattern = r"Age:\s*(\d+)"


#         # Extract Pilot Information
#         pilot_match_start = re.search(pilot_section_start_pattern, full_text)
#         if pilot_match_start:
#             pilot_text_start_index = pilot_match_start.end()
#             pilot_text_end_index = len(full_text) # Default to end of text

#             # Find the end of the pilot section based on keywords
#             # Find the first occurrence of any end keyword after the start of the pilot section
#             for keyword in section_end_keywords:
#                 keyword_match = re.search(re.escape(keyword), full_text[pilot_text_start_index:], re.IGNORECASE)
#                 if keyword_match:
#                     current_end_index = pilot_text_start_index + keyword_match.start()
#                     if current_end_index < pilot_text_end_index:
#                         pilot_text_end_index = current_end_index
#                     # Break after finding the first end keyword
#                     break


#             pilot_text = full_text[pilot_text_start_index:pilot_text_end_index].strip()
#             result['pilot_text'] = pilot_text
#             # print("\n--- Pilot Text from PDF ---")
#             # print(pilot_text)
#             # print("----------------------------\n")

#             # Extract Age and Flight Time from collected pilot text
#             if pilot_text:
#                 pilot_ages = re.findall(age_pattern, pilot_text)
#                 # Adjusted regex to capture text after "Flight Time:" and handle potential variations
#                 pilot_flight_times = re.findall(flight_time_pattern + r"\s*(.*)", pilot_text, re.S)

#                 if pilot_ages:
#                     result['pilot age'] = pilot_ages[0]
#                 else:
#                      result['pilot age'] = "Not Available" # Set if section found but age not

#                 if pilot_flight_times:
#                     result['pilot flight experience'] = " ".join(pilot_flight_times).strip()
#                 else:
#                      result['pilot flight experience'] = "Not Available" # Set if section found but flight time not
#         return result

#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading the PDF from {pdf_url}: {e}")
#         # Set values to "Error processing PDF" for rows with download errors
#         result['pilot age'] = "Error processing PDF"
#         result['pilot flight experience'] = "Error processing PDF"
#         return result

#     except Exception as e:
#         print(f"An error occurred while processing {pdf_url}: {e}")
#         # Set values to "Error processing PDF" for rows with other processing errors
#         result['pilot age'] = "Error processing PDF"
#         result['pilot flight experience'] = "Error processing PDF"
#         return result

# # --- How to use the function ---
# # Replace with the URL you want to test
# test_url = 'https://data.ntsb.gov/carol-repgen/api/Aviation/ReportMain/GenerateNewestReport/193179/pdf'

# # Process the URL and get the results
# extracted_data = process_single_pdf_url(test_url)

# # Print the extracted data
# print("\nExtracted Data:")
# for key, value in extracted_data.items():
#     print(f"{key}: {value}")


--- Full Text from PDF ---
Page 1 of 7
Aviation Investigation Final Report
Location: Lake Erie Beach, New York Accident Number: DCA23LA465
Date & Time: September 28, 2023, 08:57 UTC Registration: N804AN
Aircraft: Boeing 787-8 Aircraft Damage: None
Defining Event: Turbulence encounter Injuries: 1 Serious, 239 None
Flight Conducted Under: Part 121: Air carrier - Scheduled
Analysis 
American Airlines flight 160 encountered turbulence during climb to cruise while enroute to 
Athens International Airport (ATH), Spata, Greece from O’Hare International Airport (ORD), 
Chicago, Illinois resulting in a flight attendant (FA) being seriously injured.
The captain reported having completed a briefing with the cabin crew prior to the flight where 
he discussed turbulence precautions and pointed out areas specifically where they expected 
the flight would encounter turbulence. 
During climb to the cruise altitude of flight level (FL) 370 [37,000 feet], the flight crew reported 
that air traffic cont

In [ ]:
# Create the new columns if they don't exist
if 'PilotFlightExp(Total,AllAircraft)' not in df.columns:
    df['PilotFlightExp(Total,AllAircraft)'] = None
if 'PilotFlightExp(Total,ThisMakeAndModel)' not in df.columns:
    df['PilotFlightExp(Total,ThisMakeAndModel)'] = None

# Define regex patterns to extract the desired information
total_hours_pattern = r"(\d+)\s*hours\s*\(Total,\s*all\s*aircraft\)"
make_model_hours_pattern = r"(\d+)\s*hours\s*\(Total,\s*this\s*make\s*and\s*model\)"

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    pilot_exp = str(row['PilotFlightExp']) # Ensure the value is a string

    # Extract Total, all aircraft hours
    total_match = re.search(total_hours_pattern, pilot_exp)
    if total_match:
        df.at[index, 'PilotFlightExp(Total,AllAircraft)'] = total_match.group(1).strip()
    else:
        df.at[index, 'PilotFlightExp(Total,AllAircraft)'] = "Information Not Available"

    # Extract Total, this make and model hours
    make_model_match = re.search(make_model_hours_pattern, pilot_exp)
    if make_model_match:
        df.at[index, 'PilotFlightExp(Total,ThisMakeAndModel)'] = make_model_match.group(1).strip()
    else:
        df.at[index, 'PilotFlightExp(Total,ThisMakeAndModel)'] = "Information Not Available"

# Display the updated columns
display(df[['PilotFlightExp', 'PilotFlightExp(Total,AllAircraft)', 'PilotFlightExp(Total,ThisMakeAndModel)']].head())

/tmp/ipython-input-226110471.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'PilotFlightExp(Total,AllAircraft)'] = total_match.group(1).strip()
/tmp/ipython-input-226110471.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5979' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'PilotFlightExp(Total,ThisMakeAndModel)'] = make_model_match.group(1).strip()


,PilotFlightExp,"PilotFlightExp(Total,AllAircraft)","PilotFlightExp(Total,ThisMakeAndModel)"
0,"13000 hours (Total, all aircraft), 5979 hours ...",13000,5979
1,"6842 hours (Total, all aircraft), 4745 hours (...",6842,4745
2,Information Not in PDF,Information Not Available,Information Not Available
3,"13000 hours (Total, all aircraft), 4700 hours ...",13000,4700
4,"12920 hours (Total, all aircraft), 1844 hours ...",12920,1844


In [ ]:
# # Import parsed information into Google Sheet
# # Open the spreadsheet and the desired sheet
# spreadsheet_name = 'ext_far121_ntsb' # Replace with your spreadsheet name
# sheet_name = 'Sheet1' # Replace with your sheet name

# try:
#     sh = gc.open(spreadsheet_name)
#     worksheet = sh.worksheet(sheet_name)

#     # Select only the columns to be updated
#     columns_to_update = df[['PilotFlightExp(Total,AllAircraft)', 'PilotFlightExp(Total,ThisMakeAndModel)']].copy()

#     # Replace None and NaN values with "Information Not Available" for Google Sheet compatibility
#     columns_to_update_cleaned = columns_to_update.fillna("Information Not Available")
#     columns_to_update_cleaned = columns_to_update_cleaned.replace({None: "Information Not Available"})


#     # Convert DataFrame of columns to update to a list of lists for gspread
#     # Include headers for the columns
#     data_to_upload = [columns_to_update_cleaned.columns.values.tolist()] + columns_to_update_cleaned.values.tolist()

#     # Determine the starting cell for the columns (AB1)
#     start_cell = 'AB1'

#     # Update the sheet with only the specified columns
#     worksheet.update(values=data_to_upload, range_name=start_cell)

#     print(f"Successfully updated 'PilotFlightExp(Total,AllAircraft)' and 'PilotFlightExp(Total,ThisMakeAndModel)' columns in the Google Sheet starting at cell {start_cell}.")

# except gspread.SpreadsheetNotFound:
#     print(f"Spreadsheet '{spreadsheet_name}' not found.")
# except gspread.WorksheetNotFound:
#     print(f"Worksheet '{sheet_name}' not found.")
# except Exception as e:
#     print(f"An error occurred while updating the Google Sheet: {e}")

Successfully updated 'PilotFlightExp(Total,AllAircraft)' and 'PilotFlightExp(Total,ThisMakeAndModel)' columns in the Google Sheet starting at cell AB1.
